In [1]:
# Prerequisites
## !pip install datasets
## !pip install imbalanced-learn

# References
# https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023

In [2]:
#from datasets import load_dataset

In [3]:
# dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)
# print(dataset["full"][0])

In [4]:
# dataset["full"].to_csv("amazon_reviews.csv", index=False)

In [5]:
import pandas as pd
import numpy as np

In [6]:
## Load the dataset
data = pd.read_csv("amazon_reviews.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'amazon_reviews.csv'

In [ ]:
data = data[['text', 'rating']]
data.head()

,text,rating
0,This spray is really nice. It smells really go...,5.0
1,"This product does what I need it to do, I just...",4.0
2,"Smells good, feels great!",5.0
3,Felt synthetic,1.0
4,Love it,5.0


In [ ]:
data.shape

(701528, 2)

In [ ]:
## Missing Values
data.isnull().sum()

text      212
rating      0
dtype: int64

In [ ]:
## Checking null records
data[data['text'].isnull()]

,text,rating
9078,NaN,5.0
15159,NaN,5.0
28358,NaN,5.0
29389,NaN,4.0
34286,NaN,3.0
...,...,...
693459,NaN,4.0
700225,NaN,2.0
700357,NaN,5.0
700744,NaN,3.0


In [ ]:
## Dropping null records
data = data.dropna()
data[data['text'].isnull()]

,text,rating


In [ ]:
## Checking the dependent variable
data['rating'].unique()

array([5., 4., 1., 3., 2.])

In [ ]:
## Checking if data is balanced
data['rating'].value_counts()

rating
5.0    420566
1.0    102067
4.0     79357
3.0     56294
2.0     43032
Name: count, dtype: int64

Imbalance dataset :(